# Lab 2-3: KServe 모델 배포

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 50분 |
| **난이도** | ⭐⭐⭐ |
| **목표** | KServe를 사용한 프로덕션 모델 서빙 |

## 🎯 학습 목표

- KServe InferenceService 이해
- S3에서 MLflow 모델 로드
- REST API를 통한 추론 테스트

---

## 1. 환경 설정

### 1.1 라이브러리 Import

In [ ]:
# ============================================================
# 1.1 라이브러리 Import
# ============================================================
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import requests
import json
import time
import base64

print("✅ 라이브러리 Import 완료")

### 1.2 환경 변수 설정

⚠️ **중요**: AWS 자격증명을 본인 값으로 수정하세요!

In [ ]:
# ============================================================
# 1.2 환경 변수 설정
# ============================================================

import os


# ⚠️ 본인의 사용자 번호로 변경!
USER_NUM = "YOUR_USER_NUM"

# AWS 자격증명 (⚠️ 본인 값으로 수정!)
AWS_ACCESS_KEY_ID = "YOUR_ACCESS_KEY_HERE"        # 수정 필요!
AWS_SECRET_ACCESS_KEY = "YOUR_SECRET_KEY_HERE"    # 수정 필요!
AWS_REGION = "ap-northeast-2"


# 네임스페이스 자동 감지 (⚠️ 수동 설정 금지!)
def get_current_namespace():
    """Kubeflow Notebook의 현재 네임스페이스 자동 감지"""
    try:
        with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        # 로컬 환경에서는 USER_NUM 기반 동적 생성
        user_num = os.getenv('USER_NUM', {USER_NUM})
        return f"kubeflow-user{user_num}"

# 환경 변수 읽기
NAMESPACE = get_current_namespace()

# S3 버킷 설정 (사용자별 동적 생성)
S3_BUCKET = f"mlops-training-user{USER_NUM}"

# 모델 설정
MODEL_NAME = f"california-model-user{USER_NUM}"

print("="*60)
print("  환경 설정")
print("="*60)
print(f"  📁 네임스페이스: {NAMESPACE}")
print(f"📦 S3 버킷: {S3_BUCKET}")
print(f"👤 사용자 번호: {USER_NUM}")
print(f"  🤖 모델명: {MODEL_NAME}")
print(f"  🌏 리전: {AWS_REGION}")

if AWS_ACCESS_KEY_ID == "YOUR_ACCESS_KEY_HERE":
    print("\n⚠️  경고: AWS 자격증명을 본인 값으로 수정하세요!")

### 1.3 Kubernetes 클라이언트 연결

In [ ]:
# ============================================================
# 1.3 Kubernetes 클라이언트 연결
# ============================================================
print("="*60)
print("  Kubernetes 클라이언트 연결")
print("="*60)

try:
    # In-cluster 설정 로드 (Kubeflow Notebook 환경)
    config.load_incluster_config()
    
    # API 클라이언트 생성
    core_v1 = client.CoreV1Api()
    custom_api = client.CustomObjectsApi()
    
    print(f"  🏢 환경: Kubeflow Notebook (Kubernetes 클러스터 내부)")
    print(f"  📁 네임스페이스: {NAMESPACE}")
    print(f"  ✅ Kubernetes 클라이언트 연결 완료!")
    
except Exception as e:
    print(f"  ❌ 연결 실패: {e}")
    print(f"  💡 Kubeflow Notebook 환경에서 실행하세요.")

---

## 2. AWS 자격증명 설정

### 2.1 AWS Secret 생성

In [ ]:
# ============================================================
# 2.1 AWS Secret 생성
# ============================================================
print("="*60)
print("  AWS Secret 생성")
print("="*60)

def create_aws_secret(namespace, access_key, secret_key, region):
    """AWS S3 자격증명 Secret 생성"""
    secret_name = "aws-s3-credentials"
    
    secret = client.V1Secret(
        api_version="v1",
        kind="Secret",
        metadata=client.V1ObjectMeta(name=secret_name, namespace=namespace),
        type="Opaque",
        string_data={
            "AWS_ACCESS_KEY_ID": access_key,
            "AWS_SECRET_ACCESS_KEY": secret_key,
            "AWS_DEFAULT_REGION": region
        }
    )
    
    try:
        # 기존 Secret 삭제 (있으면)
        core_v1.delete_namespaced_secret(secret_name, namespace)
        print(f"  🗑️  기존 Secret '{secret_name}' 삭제")
    except ApiException as e:
        if e.status != 404:
            raise
    
    # Secret 생성
    core_v1.create_namespaced_secret(namespace=namespace, body=secret)
    print(f"  ✅ Secret '{secret_name}' 생성 완료")
    return secret_name

# Secret 생성 실행
secret_name = create_aws_secret(
    namespace=NAMESPACE,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    region=AWS_REGION
)

### 2.2 S3 ConfigMap 생성

In [ ]:
# ============================================================
# 2.2 S3 ConfigMap 생성
# ============================================================
print("="*60)
print("  S3 ConfigMap 생성")
print("="*60)

def create_s3_config(namespace, region):
    """S3 설정 ConfigMap 생성"""
    configmap_name = "s3-config"
    
    configmap = client.V1ConfigMap(
        api_version="v1",
        kind="ConfigMap",
        metadata=client.V1ObjectMeta(name=configmap_name, namespace=namespace),
        data={
            "S3_ENDPOINT": "s3.amazonaws.com",
            "S3_USE_HTTPS": "1",
            "AWS_REGION": region
        }
    )
    
    try:
        core_v1.delete_namespaced_config_map(configmap_name, namespace)
        print(f"  🗑️  기존 ConfigMap '{configmap_name}' 삭제")
    except ApiException as e:
        if e.status != 404:
            raise
    
    core_v1.create_namespaced_config_map(namespace=namespace, body=configmap)
    print(f"  ✅ ConfigMap '{configmap_name}' 생성 완료")
    return configmap_name

# ConfigMap 생성 실행
configmap_name = create_s3_config(NAMESPACE, AWS_REGION)

### 2.3 자격증명 확인

In [ ]:
# ============================================================
# 2.3 자격증명 확인
# ============================================================
print("="*60)
print("  자격증명 확인")
print("="*60)

def verify_credentials(namespace):
    """자격증명 존재 여부 확인"""
    # Secret 확인
    try:
        secret = core_v1.read_namespaced_secret("aws-s3-credentials", namespace)
        print(f"  ✅ Secret 'aws-s3-credentials' 존재")
        print(f"     - Keys: {list(secret.data.keys())}")
    except ApiException:
        print(f"  ❌ Secret 'aws-s3-credentials' 없음")
        return False
    
    # ConfigMap 확인
    try:
        cm = core_v1.read_namespaced_config_map("s3-config", namespace)
        print(f"  ✅ ConfigMap 's3-config' 존재")
        print(f"     - Keys: {list(cm.data.keys())}")
    except ApiException:
        print(f"  ❌ ConfigMap 's3-config' 없음")
        return False
    
    return True

verify_credentials(NAMESPACE)

### 2.4 MLflow 연결 확인 (멀티테넌트)

In [ ]:
import os
import mlflow

# 환경 변수 확인
MLFLOW_URI = os.environ.get('MLFLOW_TRACKING_URI', '')
print(f"MLFLOW_TRACKING_URI: {MLFLOW_URI or 'Not set'}")

# 환경 변수가 없으면 수동 설정
if not MLFLOW_URI:
    USER_NUM = "01"  # 본인 번호로 변경!
    MLFLOW_URI = f"http://mlflow-server.kubeflow-user{USER_NUM}.svc.cluster.local:5000"
    mlflow.set_tracking_uri(MLFLOW_URI)
    print(f"⚠️ 수동 설정: {MLFLOW_URI}")

# 연결 테스트
try:
    experiments = mlflow.search_experiments()
    print(f"✅ MLflow 연결 성공! 실험 수: {len(experiments)}")
except Exception as e:
    print(f"❌ MLflow 연결 실패: {e}")

---

## 3. InferenceService 배포

### 3.1 S3 모델 경로 설정

⚠️ **중요**: MLflow UI 또는 S3에서 확인한 **Run ID**로 수정하세요!

```bash
# S3에서 모델 경로 확인
aws s3 ls s3://mlops-training-user<YOUR_USER_NUM>/mlflow-artifacts/ --recursive | grep "MLmodel"
```

In [ ]:
# ============================================================
# 3.1 S3 모델 경로 설정
# ============================================================
print("="*60)
print("  모델 경로 설정")
print("="*60)

# ⚠️ 중요: 아래 값들을 본인의 MLflow Run 정보로 수정하세요!
RUN_ID = "YOUR_RUN_ID_HERE"  # MLflow Run ID (수정 필요!)

# S3 전체 경로 구성
# ⚠️ 중요: mlflow-artifacts: 프로토콜이 아닌 s3:// 전체 경로 사용!
STORAGE_URI = f"s3://{S3_BUCKET}/mlflow-artifacts/{RUN_ID}/artifacts/model"

print(f"  📦 Storage URI: {STORAGE_URI}")

if RUN_ID == "YOUR_RUN_ID_HERE":
    print("\n⚠️  경고: RUN_ID를 실제 값으로 수정하세요!")
    print("   MLflow UI에서 확인하거나 아래 명령어로 찾을 수 있습니다:")
    print(f"   aws s3 ls s3://{S3_BUCKET}/mlflow-artifacts/ --recursive | grep MLmodel")

### 3.2 InferenceService 생성

In [ ]:
# ============================================================
# 3.2 InferenceService 생성
# ============================================================
print("="*60)
print("  InferenceService 배포")
print("="*60)

# InferenceService 스펙 정의
isvc_spec = {
    "apiVersion": "serving.kserve.io/v1beta1",
    "kind": "InferenceService",
    "metadata": {
        "name": MODEL_NAME,
        "namespace": NAMESPACE,
        "annotations": {
            # ⚠️ 중요: Istio sidecar 비활성화 (RBAC 403 에러 방지)
            "sidecar.istio.io/inject": "false"
        }
    },
    "spec": {
        "predictor": {
            "model": {
                "modelFormat": {"name": "sklearn"},
                "storageUri": STORAGE_URI,
                "resources": {
                    "requests": {"cpu": "500m", "memory": "1Gi"},
                    "limits": {"cpu": "1", "memory": "2Gi"}
                }
            }
        }
    }
}

def deploy_inferenceservice(spec, namespace):
    """InferenceService 배포"""
    model_name = spec["metadata"]["name"]
    
    # 기존 InferenceService 삭제 (있으면)
    try:
        custom_api.delete_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        print(f"  🗑️  기존 InferenceService '{model_name}' 삭제")
        time.sleep(5)  # 삭제 완료 대기
    except ApiException as e:
        if e.status != 404:
            raise
    
    # InferenceService 생성
    custom_api.create_namespaced_custom_object(
        group="serving.kserve.io",
        version="v1beta1",
        namespace=namespace,
        plural="inferenceservices",
        body=spec
    )
    print(f"  ✅ InferenceService '{model_name}' 생성 완료")

# 배포 실행
deploy_inferenceservice(isvc_spec, NAMESPACE)

### 3.3 배포 상태 모니터링

In [ ]:
# ============================================================
# 3.3 배포 상태 모니터링
# ============================================================
print("="*60)
print("  배포 상태 모니터링")
print("="*60)

def wait_for_inferenceservice(model_name, namespace, timeout=300):
    """InferenceService Ready 대기"""
    print(f"  ⏳ InferenceService '{model_name}' Ready 대기 중...")
    print(f"     (최대 {timeout}초, 보통 2-3분 소요)")
    print("")
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            isvc = custom_api.get_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                name=model_name
            )
            
            conditions = isvc.get("status", {}).get("conditions", [])
            ready = next((c for c in conditions if c.get("type") == "Ready"), None)
            
            if ready:
                status = ready.get("status")
                reason = ready.get("reason", "")
                message = ready.get("message", "")
                
                elapsed = int(time.time() - start_time)
                
                if status == "True":
                    url = isvc.get("status", {}).get("url", "")
                    print(f"\n  ✅ InferenceService Ready! ({elapsed}초 소요)")
                    print(f"     URL: {url}")
                    return True
                elif status == "False":
                    print(f"  ❌ Status: {status} | Reason: {reason}")
                    if "ModelLoadFailed" in reason:
                        print(f"     Message: {message[:100]}...")
                        return False
                else:
                    print(f"  ⏳ Status: {status} | Reason: {reason} ({elapsed}초)")
            
        except ApiException as e:
            print(f"  ⚠️ 상태 확인 실패: {e.reason}")
        
        time.sleep(10)
    
    print(f"\n  ❌ 타임아웃: {timeout}초 초과")
    return False

# 모니터링 실행
is_ready = wait_for_inferenceservice(MODEL_NAME, NAMESPACE)

---

## 4. 추론 테스트

### 4.1 InferenceService 상태 확인

In [ ]:
# ============================================================
# 4.1 InferenceService 상태 확인
# ============================================================
print("="*60)
print("  InferenceService 상태 확인")
print("="*60)

def get_inferenceservice_status(model_name, namespace):
    """InferenceService 상태 조회"""
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        url = isvc.get("status", {}).get("url", "N/A")
        conditions = isvc.get("status", {}).get("conditions", [])
        ready = next((c for c in conditions if c.get("type") == "Ready"), {})
        
        print(f"  📊 모델명: {model_name}")
        print(f"  📁 네임스페이스: {namespace}")
        print(f"  🔗 URL: {url}")
        print(f"  ✅ Ready: {ready.get('status', 'Unknown')}")
        
        return ready.get('status') == 'True'
        
    except ApiException as e:
        print(f"  ❌ 조회 실패: {e.reason}")
        return False

is_ready = get_inferenceservice_status(MODEL_NAME, NAMESPACE)

### 4.2 클러스터 내부 추론 테스트

In [ ]:
# ============================================================
# 4.2 클러스터 내부 추론 테스트
# ============================================================
print("="*60)
print("  클러스터 내부 추론 테스트")
print("="*60)

# ⚠️ California Housing 데이터셋: 8개 특성
# [MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude]
test_data = {
    "instances": [
        [3.5, 25.0, 5.5, 1.1, 1500.0, 3.0, 37.5, -122.0]
    ]
}

def test_inference_internal(model_name, namespace, data):
    """클러스터 내부에서 추론 테스트"""
    # KServe 내부 URL 구성
    url = f"http://{model_name}-predictor.{namespace}.svc.cluster.local/v1/models/{model_name}:predict"
    
    print(f"  🔗 URL: {url}")
    print(f"  📤 입력 데이터: {data}")
    
    try:
        response = requests.post(
            url,
            json=data,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"  ✅ 추론 성공!")
            print(f"  📥 예측 결과: {result}")
            return result
        else:
            print(f"  ❌ 추론 실패: HTTP {response.status_code}")
            print(f"      {response.text}")
            return None
            
    except requests.exceptions.Timeout:
        print(f"  ❌ 타임아웃: 서버 응답 없음")
        return None
    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ 연결 실패: {e}")
        return None
    except Exception as e:
        print(f"  ❌ 에러: {e}")
        return None

# 테스트 실행
result = test_inference_internal(MODEL_NAME, NAMESPACE, test_data)

if result:
    predictions = result.get("predictions", [])
    print("")
    print("="*60)
    if predictions:
        # California Housing 타겟은 $100,000 단위
        print(f"  🏠 예측된 주택 가격: ${predictions[0] * 100000:,.0f}")
    print("="*60)

### 4.3 다양한 입력 테스트

In [ ]:
# ============================================================
# 4.3 다양한 입력 테스트
# ============================================================
print("="*60)
print("  다양한 입력 테스트")
print("="*60)

# 여러 샘플 테스트
# California Housing 특성: [MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude]
test_samples = {
    "instances": [
        [8.3252, 41.0, 6.98, 1.02, 322.0, 2.56, 37.88, -122.23],   # 고가 지역 (SF Bay Area)
        [3.5, 25.0, 5.5, 1.1, 1500.0, 3.0, 37.5, -122.0],          # 중간 지역
        [1.5, 10.0, 4.0, 1.0, 3000.0, 4.0, 34.0, -118.0],          # 저가 지역 (LA)
    ]
}

labels = ["고가 지역 (SF Bay Area)", "중간 지역", "저가 지역 (LA)"]

result = test_inference_internal(MODEL_NAME, NAMESPACE, test_samples)

if result:
    predictions = result.get("predictions", [])
    print("")
    print("  📊 예측 결과 요약:")
    print("  " + "-"*40)
    for i, (pred, label) in enumerate(zip(predictions, labels)):
        print(f"  {label}: ${pred * 100000:,.0f}")
    print("  " + "-"*40)

---

## 5. 정리 및 확인

In [ ]:
# ============================================================
# 5. 최종 상태 확인
# ============================================================
print("="*60)
print("  Lab 2-3 완료 체크리스트")
print("="*60)

def final_check(namespace, model_name):
    """최종 상태 확인"""
    checks = []
    
    # Secret 확인
    try:
        core_v1.read_namespaced_secret("aws-s3-credentials", namespace)
        checks.append(("AWS Secret 생성", True))
    except:
        checks.append(("AWS Secret 생성", False))
    
    # ConfigMap 확인
    try:
        core_v1.read_namespaced_config_map("s3-config", namespace)
        checks.append(("S3 ConfigMap 생성", True))
    except:
        checks.append(("S3 ConfigMap 생성", False))
    
    # InferenceService 확인
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        conditions = isvc.get("status", {}).get("conditions", [])
        ready = next((c for c in conditions if c.get("type") == "Ready"), {})
        is_ready = ready.get("status") == "True"
        checks.append(("InferenceService Ready", is_ready))
    except:
        checks.append(("InferenceService Ready", False))
    
    # 결과 출력
    all_passed = True
    for check_name, passed in checks:
        status = "✅" if passed else "❌"
        print(f"  {status} {check_name}")
        if not passed:
            all_passed = False
    
    print("")
    if all_passed:
        print("  🎉 Lab 2-3 완료!")
    else:
        print("  ⚠️ 일부 항목이 완료되지 않았습니다.")
        print("     TROUBLESHOOTING.md를 참조하세요.")
    
    return all_passed

final_check(NAMESPACE, MODEL_NAME)

---

## 📚 다음 단계

Lab 2-3이 완료되었습니다! 다음 Lab으로 진행하세요:

- **Day 3 Lab 3-1**: Kubeflow Pipelines
- **Day 3 Lab 3-2**: 모니터링 및 CI/CD